<a href="https://colab.research.google.com/github/ancestor9/2025_Spring_Data-Management/blob/main/week_10/03_Data_Sales_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faker --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.9 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel, Field
from typing import Optional, List
from decimal import Decimal
from datetime import datetime
from faker import Faker
import random

# Faker 인스턴스 생성 (한국어 데이터)
fake = Faker('ko_KR')

class Customer(BaseModel):
    """고객 정보 모델"""
    id: Optional[int] = Field(None, description="고객 ID")
    name: str = Field(..., min_length=1, max_length=100, description="고객 이름")
    age: int = Field(..., ge=0, le=150, description="고객 나이")
    address: str = Field(..., min_length=1, max_length=500, description="고객 주소")
    created_at: Optional[datetime] = Field(default_factory=datetime.now, description="생성 일시")

class Product(BaseModel):
    """상품 정보 모델"""
    id: Optional[int] = Field(None, description="상품 ID")
    name: str = Field(..., min_length=1, max_length=200, description="상품명")
    product_type: str = Field(..., min_length=1, max_length=50, description="상품 유형")
    price: Decimal = Field(..., ge=0, decimal_places=2, description="가격")
    manufacturer: str = Field(..., min_length=1, max_length=100, description="제조사")
    created_at: Optional[datetime] = Field(default_factory=datetime.now, description="생성 일시")

# 상품 카테고리와 제조사 목록
PRODUCT_TYPES = [
    "스마트폰", "노트북", "태블릿", "스마트워치", "이어폰",
    "냉장고", "세탁기", "에어컨", "TV", "청소기",
    "전자레인지", "오븐", "커피머신", "블렌더", "토스터"
]

MANUFACTURERS = [
    "삼성전자", "LG전자", "애플", "소니", "파나소닉",
    "필립스", "다이슨", "보쉬", "샤오미", "화웨이",
    "델", "HP", "레노버", "에이서", "ASUS"
]

# 상품명 생성 함수
def generate_product_name(product_type: str) -> str:
    adjectives = ["프리미엄", "스마트", "프로", "울트라", "맥스", "플러스", "에코", "그랜드", "뉴", "슬림"]
    models = ["X1", "S2", "Pro", "Max", "Plus", "Ultra", "2024", "Premium", "Elite", "Advanced"]

    return f"{random.choice(adjectives)} {product_type} {random.choice(models)}"

# 가짜 고객 데이터 생성 함수
def create_fake_customers(count: int) -> List[Customer]:
    customers = []
    for i in range(1, count + 1):
        customer = Customer(
            id=i,
            name=fake.name(),
            age=random.randint(18, 80),
            address=fake.address(),
            created_at=fake.date_time_between(start_date='-1y', end_date='now')
        )
        customers.append(customer)
    return customers

# 가짜 상품 데이터 생성 함수
def create_fake_products(count: int) -> List[Product]:
    products = []
    for i in range(1, count + 1):
        product_type = random.choice(PRODUCT_TYPES)

        # 상품 타입에 따른 가격 범위 설정
        if product_type in ["스마트폰", "노트북", "TV"]:
            min_price, max_price = 500000, 3000000
        elif product_type in ["냉장고", "세탁기", "에어컨"]:
            min_price, max_price = 400000, 2000000
        else:
            min_price, max_price = 50000, 500000

        product = Product(
            id=i,
            name=generate_product_name(product_type),
            product_type=product_type,
            price=Decimal(str(round(random.uniform(min_price, max_price), -3))),  # 천원 단위로 반올림
            manufacturer=random.choice(MANUFACTURERS),
            created_at=fake.date_time_between(start_date='-1y', end_date='now')
        )
        products.append(product)
    return products

# 데이터 생성
customers = create_fake_customers(10)
products = create_fake_products(100)

# 결과 출력
print("=== 고객 목록 (10명) ===")
for customer in customers:
    print(f"ID: {customer.id}, 이름: {customer.name}, 나이: {customer.age}, 주소: {customer.address}")

print("\n=== 상품 목록 (처음 10개) ===")
for product in products[:10]:
    print(f"ID: {product.id}, 상품명: {product.name}, 유형: {product.product_type}, "
          f"가격: {product.price:,}원, 제조사: {product.manufacturer}")

# JSON 형태로 저장하기
import json

def save_to_json(customers: List[Customer], products: List[Product]):
    with open('customers.json', 'w', encoding='utf-8') as f:
        json.dump([customer.model_dump() for customer in customers], f, ensure_ascii=False, indent=2, default=str)

    with open('products.json', 'w', encoding='utf-8') as f:
        json.dump([product.model_dump() for product in products], f, ensure_ascii=False, indent=2, default=str)

save_to_json(customers, products)

# 데이터프레임으로 변환 (분석용)
import pandas as pd

def to_dataframe(customers: List[Customer], products: List[Product]):
    customers_df = pd.DataFrame([customer.model_dump() for customer in customers])
    products_df = pd.DataFrame([product.model_dump() for product in products])

    return customers_df, products_df

customers_df, products_df = to_dataframe(customers, products)

# 간단한 통계 정보
print("\n=== 고객 통계 ===")
print(f"평균 나이: {customers_df['age'].mean():.1f}세")
print(f"최연소: {customers_df['age'].min()}세")
print(f"최고령: {customers_df['age'].max()}세")

print("\n=== 상품 통계 ===")
print(f"평균 가격: {products_df['price'].mean():,.0f}원")
print(f"최저가: {products_df['price'].min():,.0f}원")
print(f"최고가: {products_df['price'].max():,.0f}원")

print("\n=== 상품 유형별 분포 ===")
print(products_df['product_type'].value_counts())

print("\n=== 제조사별 분포 ===")
print(products_df['manufacturer'].value_counts().head(10))

=== 고객 목록 (10명) ===
ID: 1, 이름: 박정호, 나이: 64, 주소: 제주특별자치도 안양시 학동705길 913 (지훈김면)
ID: 2, 이름: 김정훈, 나이: 26, 주소: 경기도 과천시 강남대3로 530-78 (도현김마을)
ID: 3, 이름: 박정숙, 나이: 73, 주소: 강원도 안산시 단원구 오금거리 지하873
ID: 4, 이름: 이지은, 나이: 23, 주소: 부산광역시 영등포구 잠실3길 656-20 (지훈고김동)
ID: 5, 이름: 최경희, 나이: 66, 주소: 경상남도 청주시 청원구 가락로 763-71 (예준황리)
ID: 6, 이름: 최미경, 나이: 23, 주소: 광주광역시 광진구 서초중앙로 753
ID: 7, 이름: 김예진, 나이: 35, 주소: 전라남도 안양시 동안구 도산대거리 116-38 (승현이동)
ID: 8, 이름: 김성민, 나이: 51, 주소: 대전광역시 용산구 압구정2가 280-51
ID: 9, 이름: 박정수, 나이: 64, 주소: 울산광역시 종로구 삼성44거리 263-26
ID: 10, 이름: 김정순, 나이: 63, 주소: 서울특별시 동대문구 논현가 903-51 (명숙이리)

=== 상품 목록 (처음 10개) ===
ID: 1, 상품명: 프리미엄 전자레인지 Elite, 유형: 전자레인지, 가격: 56,000.0원, 제조사: 에이서
ID: 2, 상품명: 울트라 태블릿 2024, 유형: 태블릿, 가격: 204,000.0원, 제조사: 에이서
ID: 3, 상품명: 플러스 청소기 Advanced, 유형: 청소기, 가격: 91,000.0원, 제조사: LG전자
ID: 4, 상품명: 울트라 청소기 Pro, 유형: 청소기, 가격: 337,000.0원, 제조사: ASUS
ID: 5, 상품명: 울트라 이어폰 Elite, 유형: 이어폰, 가격: 216,000.0원, 제조사: 레노버
ID: 6, 상품명: 뉴 태블릿 Pro, 유형: 태블릿, 가격: 123,000.0원, 제조사: 삼성전자
ID: 7, 상품명: 프리미엄 이어폰 X1, 유형: 이어폰, 

In [3]:
! pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


In [4]:
from pydantic import BaseModel, Field
from typing import Optional, List
from decimal import Decimal
from datetime import datetime
from faker import Faker
import random
import gradio as gr
import pandas as pd
import json
from uuid import uuid4

# Faker 인스턴스 생성 (한국어 데이터)
fake = Faker('ko_KR')

# Pydantic 모델 정의
class Customer(BaseModel):
    id: Optional[int] = Field(None, description="고객 ID")
    name: str = Field(..., min_length=1, max_length=100, description="고객 이름")
    age: int = Field(..., ge=0, le=150, description="고객 나이")
    address: str = Field(..., min_length=1, max_length=500, description="고객 주소")
    created_at: Optional[datetime] = Field(default_factory=datetime.now, description="생성 일시")

class Product(BaseModel):
    id: Optional[int] = Field(None, description="상품 ID")
    name: str = Field(..., min_length=1, max_length=200, description="상품명")
    product_type: str = Field(..., min_length=1, max_length=50, description="상품 유형")
    price: Decimal = Field(..., ge=0, decimal_places=2, description="가격")
    manufacturer: str = Field(..., min_length=1, max_length=100, description="제조사")
    created_at: Optional[datetime] = Field(default_factory=datetime.now, description="생성 일시")

class PurchaseItem(BaseModel):
    product_id: int = Field(..., description="상품 ID")
    product_name: str = Field(..., description="상품명")
    quantity: int = Field(..., ge=1, description="수량")
    unit_price: Decimal = Field(..., ge=0, description="단가")
    subtotal: Decimal = Field(..., ge=0, description="소계")

class Purchase(BaseModel):
    purchase_id: str = Field(default_factory=lambda: str(uuid4()), description="구매 ID")
    customer_id: int = Field(..., description="고객 ID")
    customer_name: str = Field(..., description="고객 이름")
    items: List[PurchaseItem] = Field(..., description="구매 상품 목록")
    total_amount: Decimal = Field(..., ge=0, description="총 구매금액")
    purchase_date: datetime = Field(default_factory=datetime.now, description="구매 일시")

# 데이터 생성 함수들
PRODUCT_TYPES = ["스마트폰", "노트북", "태블릿", "스마트워치", "이어폰",
                 "냉장고", "세탁기", "에어컨", "TV", "청소기"]

MANUFACTURERS = ["삼성전자", "LG전자", "애플", "소니", "파나소닉",
                 "필립스", "다이슨", "보쉬", "샤오미", "화웨이"]

def generate_product_name(product_type: str) -> str:
    adjectives = ["프리미엄", "스마트", "프로", "울트라", "맥스"]
    models = ["X1", "S2", "Pro", "Max", "Plus", "2024"]
    return f"{random.choice(adjectives)} {product_type} {random.choice(models)}"

def create_fake_customers(count: int) -> List[Customer]:
    customers = []
    for i in range(1, count + 1):
        customer = Customer(
            id=i,
            name=fake.name(),
            age=random.randint(18, 80),
            address=fake.address()
        )
        customers.append(customer)
    return customers

def create_fake_products(count: int) -> List[Product]:
    products = []
    for i in range(1, count + 1):
        product_type = random.choice(PRODUCT_TYPES)

        if product_type in ["스마트폰", "노트북", "TV"]:
            min_price, max_price = 500000, 3000000
        elif product_type in ["냉장고", "세탁기", "에어컨"]:
            min_price, max_price = 400000, 2000000
        else:
            min_price, max_price = 50000, 500000

        product = Product(
            id=i,
            name=generate_product_name(product_type),
            product_type=product_type,
            price=Decimal(str(round(random.uniform(min_price, max_price), -3))),
            manufacturer=random.choice(MANUFACTURERS)
        )
        products.append(product)
    return products

# 초기 데이터 생성
customers = create_fake_customers(10)
products = create_fake_products(100)
purchases = []

# Gradio 애플리케이션 구현
class PurchaseSimulator:
    def __init__(self):
        self.cart = []
        self.current_customer = None

    def get_customer_list(self):
        return [(f"{c.name} (ID: {c.id})", c.id) for c in customers]

    def get_product_list(self):
        product_options = []
        for p in products:
            product_options.append(
                (f"{p.name} - {p.product_type} - {p.price:,}원 ({p.manufacturer})", p.id)
            )
        return product_options

    def select_customer(self, customer_id):
        self.current_customer = next((c for c in customers if c.id == customer_id), None)
        if self.current_customer:
            return f"선택된 고객: {self.current_customer.name} (나이: {self.current_customer.age}세)"
        return "고객을 선택해주세요."

    def add_to_cart(self, product_id, quantity):
        if not self.current_customer:
            return "먼저 고객을 선택해주세요.", self.display_cart()

        product = next((p for p in products if p.id == product_id), None)
        if not product:
            return "상품을 찾을 수 없습니다.", self.display_cart()

        # 장바구니에 이미 있는 상품인지 확인
        existing_item = next((item for item in self.cart if item['product_id'] == product_id), None)

        if existing_item:
            existing_item['quantity'] += quantity
            existing_item['subtotal'] = existing_item['unit_price'] * existing_item['quantity']
        else:
            self.cart.append({
                'product_id': product.id,
                'product_name': product.name,
                'quantity': quantity,
                'unit_price': product.price,
                'subtotal': product.price * quantity
            })

        return f"{product.name} {quantity}개가 장바구니에 추가되었습니다.", self.display_cart()

    def display_cart(self):
        if not self.cart:
            return pd.DataFrame(columns=['상품명', '수량', '단가', '소계'])

        cart_df = pd.DataFrame(self.cart)
        cart_df = cart_df[['product_name', 'quantity', 'unit_price', 'subtotal']]
        cart_df.columns = ['상품명', '수량', '단가', '소계']
        cart_df['단가'] = cart_df['단가'].apply(lambda x: f"{x:,}원")
        cart_df['소계'] = cart_df['소계'].apply(lambda x: f"{x:,}원")
        return cart_df

    def complete_purchase(self):
        if not self.current_customer:
            return "먼저 고객을 선택해주세요.", self.display_cart(), self.display_purchases()

        if not self.cart:
            return "장바구니가 비어있습니다.", self.display_cart(), self.display_purchases()

        # PurchaseItem 객체 생성
        purchase_items = []
        for item in self.cart:
            purchase_item = PurchaseItem(
                product_id=item['product_id'],
                product_name=item['product_name'],
                quantity=item['quantity'],
                unit_price=item['unit_price'],
                subtotal=item['subtotal']
            )
            purchase_items.append(purchase_item)

        # 총액 계산
        total_amount = sum(item['subtotal'] for item in self.cart)

        # Purchase 객체 생성
        purchase = Purchase(
            customer_id=self.current_customer.id,
            customer_name=self.current_customer.name,
            items=purchase_items,
            total_amount=total_amount
        )

        purchases.append(purchase)

        # 구매 완료 후 장바구니 초기화
        result_message = f"구매가 완료되었습니다! 구매 ID: {purchase.purchase_id}, 총액: {total_amount:,}원"
        self.cart = []

        return result_message, self.display_cart(), self.display_purchases()

    def display_purchases(self):
        if not purchases:
            return pd.DataFrame(columns=['구매ID', '고객명', '상품수', '총액', '구매일시'])

        purchase_data = []
        for p in purchases:
            purchase_data.append({
                '구매ID': p.purchase_id[:8] + "...",  # UUID 앞부분만 표시
                '고객명': p.customer_name,
                '상품수': len(p.items),
                '총액': f"{p.total_amount:,}원",
                '구매일시': p.purchase_date.strftime("%Y-%m-%d %H:%M:%S")
            })

        return pd.DataFrame(purchase_data)

    def save_purchases(self):
        if not purchases:
            return "저장할 구매 내역이 없습니다."

        # Pydantic 모델을 JSON으로 변환
        purchases_dict = [p.model_dump() for p in purchases]

        # Decimal과 datetime을 직렬화 가능한 형태로 변환
        for p in purchases_dict:
            p['total_amount'] = float(p['total_amount'])
            p['purchase_date'] = p['purchase_date'].isoformat()
            for item in p['items']:
                item['unit_price'] = float(item['unit_price'])
                item['subtotal'] = float(item['subtotal'])

        with open('purchases.json', 'w', encoding='utf-8') as f:
            json.dump(purchases_dict, f, ensure_ascii=False, indent=2)

        return f"{len(purchases)}개의 구매 내역이 purchases.json 파일로 저장되었습니다."

# Gradio 인터페이스 생성
simulator = PurchaseSimulator()

with gr.Blocks(title="상품 구매 시뮬레이션") as demo:
    gr.Markdown("# 상품 구매 시뮬레이션")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 1. 고객 선택")
            customer_dropdown = gr.Dropdown(
                choices=simulator.get_customer_list(),
                label="고객 선택",
                type="value"
            )
            customer_info = gr.Textbox(label="선택된 고객 정보", interactive=False)

        with gr.Column(scale=2):
            gr.Markdown("### 2. 상품 선택")
            product_dropdown = gr.Dropdown(
                choices=simulator.get_product_list(),
                label="상품 선택",
                type="value"
            )
            quantity_input = gr.Number(value=1, minimum=1, label="수량", precision=0)
            add_button = gr.Button("장바구니에 추가")

    with gr.Row():
        gr.Markdown("### 3. 장바구니")

    cart_display = gr.Dataframe(
        headers=['상품명', '수량', '단가', '소계'],
        label="장바구니 내역"
    )

    with gr.Row():
        purchase_button = gr.Button("구매 완료", variant="primary")
        save_button = gr.Button("구매 내역 저장")

    status_message = gr.Textbox(label="상태 메시지", interactive=False)

    gr.Markdown("### 4. 구매 내역")
    purchases_display = gr.Dataframe(
        headers=['구매ID', '고객명', '상품수', '총액', '구매일시'],
        label="전체 구매 내역"
    )

    # 이벤트 핸들러 연결
    customer_dropdown.change(
        simulator.select_customer,
        inputs=[customer_dropdown],
        outputs=[customer_info]
    )

    add_button.click(
        simulator.add_to_cart,
        inputs=[product_dropdown, quantity_input],
        outputs=[status_message, cart_display]
    )

    purchase_button.click(
        simulator.complete_purchase,
        outputs=[status_message, cart_display, purchases_display]
    )

    save_button.click(
        simulator.save_purchases,
        outputs=[status_message]
    )

# 애플리케이션 실행
if __name__ == "__main__":
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://05c43ce442fd53db09.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
